In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [4]:
#get all the airplines' name
url='https://www.airlinequality.com/review-pages/a-z-lounge-reviews/'
page=requests.get(url)
soup=BeautifulSoup(page.content,'html.parser')
name_section=soup.find_all('div',class_='content')
names=[]
for letter in name_section:
    airlines=letter.find_all('li')
    for airline in airlines:
        name=airline.find('a').get_text(strip=True)
        names.append(name)
print(len(names))

154


In [5]:
names

['Adria Airways',
 'Aegean Airlines',
 'Aer Lingus',
 'Aeroflot Russian Airlines',
 'Aeromexico',
 'Air Astana',
 'Air Berlin',
 'Air Canada',
 'Air Caraibes',
 'Air China',
 "Air Cote d'Ivoire",
 'Air Europa',
 'Air France',
 'Air India',
 'Air Malta',
 'Air Mauritius',
 'Air New Zealand',
 'Air Niugini',
 'Air Serbia',
 'Air Seychelles',
 'Air Tahiti Nui',
 'Air Transat',
 'AirAsia X',
 'Aircalin',
 'AIRDO',
 'Alaska Airlines',
 'Alitalia',
 'American Airlines',
 'American Express Centurion',
 'ANA All Nippon Airways',
 'Asiana Airlines',
 'ASKY Airlines',
 'Aspire by Swissport',
 'Austrian Airlines',
 'Avianca',
 'Aviator Lounge CPH',
 'Azerbaijan Airlines',
 'Bangkok Airways',
 'Biman Bangladesh Airlines',
 'Blue Islands',
 'British Airways',
 'Brussels Airlines',
 'Bulgaria Air',
 'Cathay Dragon',
 'Cathay Pacific Airways',
 'China Airlines',
 'China Eastern Airlines',
 'China Southern Airlines',
 'Comair',
 'Copa Airlines',
 'Corsair',
 'Croatia Airlines',
 'CSA Czech Airlines',


In [6]:
#transform the name format
def name_transformation(name):
    import re
    name=name.lower()
    name=re.sub(' ','-',name)
    return name

In [7]:
url_name=[name_transformation(name) for name in names]
print(url_name)

['adria-airways', 'aegean-airlines', 'aer-lingus', 'aeroflot-russian-airlines', 'aeromexico', 'air-astana', 'air-berlin', 'air-canada', 'air-caraibes', 'air-china', "air-cote-d'ivoire", 'air-europa', 'air-france', 'air-india', 'air-malta', 'air-mauritius', 'air-new-zealand', 'air-niugini', 'air-serbia', 'air-seychelles', 'air-tahiti-nui', 'air-transat', 'airasia-x', 'aircalin', 'airdo', 'alaska-airlines', 'alitalia', 'american-airlines', 'american-express-centurion', 'ana-all-nippon-airways', 'asiana-airlines', 'asky-airlines', 'aspire-by-swissport', 'austrian-airlines', 'avianca', 'aviator-lounge-cph', 'azerbaijan-airlines', 'bangkok-airways', 'biman-bangladesh-airlines', 'blue-islands', 'british-airways', 'brussels-airlines', 'bulgaria-air', 'cathay-dragon', 'cathay-pacific-airways', 'china-airlines', 'china-eastern-airlines', 'china-southern-airlines', 'comair', 'copa-airlines', 'corsair', 'croatia-airlines', 'csa-czech-airlines', 'delta-air-lines', 'dnata', 'dragonair', 'druk-air',

In [8]:
#get the page
def page_scraper(airline_name):
    url='https://www.airlinequality.com/lounge-reviews/{}/?sortby=post_date%3ADesc&pagesize=100'
    try:
        response=requests.get(url.format(airline_name))
        soup=BeautifulSoup(response.content,'html.parser')
        panel=soup.find('article',class_='comp comp_reviews-pagination querylist-pagination position-')
        if panel==None:
            page=1
        else:
            page_list=panel.find_all('a')
            page=page_list[-2].text
    except requests.RequestException as e:
        print(f'Error fecting the page {airline_name}:{e}')
        page=None
    return page

In [9]:
lounge_page={}
for airline in url_name:
    page=page_scraper(airline)
    lounge_page[airline]=page
print(lounge_page)

{'adria-airways': 1, 'aegean-airlines': 1, 'aer-lingus': 1, 'aeroflot-russian-airlines': 1, 'aeromexico': 1, 'air-astana': 1, 'air-berlin': 1, 'air-canada': '2', 'air-caraibes': 1, 'air-china': 1, "air-cote-d'ivoire": 1, 'air-europa': 1, 'air-france': 1, 'air-india': 1, 'air-malta': 1, 'air-mauritius': 1, 'air-new-zealand': 1, 'air-niugini': 1, 'air-serbia': 1, 'air-seychelles': 1, 'air-tahiti-nui': 1, 'air-transat': 1, 'airasia-x': 1, 'aircalin': 1, 'airdo': 1, 'alaska-airlines': 1, 'alitalia': 1, 'american-airlines': '2', 'american-express-centurion': 1, 'ana-all-nippon-airways': 1, 'asiana-airlines': 1, 'asky-airlines': 1, 'aspire-by-swissport': 1, 'austrian-airlines': 1, 'avianca': 1, 'aviator-lounge-cph': 1, 'azerbaijan-airlines': 1, 'bangkok-airways': 1, 'biman-bangladesh-airlines': 1, 'blue-islands': 1, 'british-airways': '5', 'brussels-airlines': 1, 'bulgaria-air': 1, 'cathay-dragon': 1, 'cathay-pacific-airways': '2', 'china-airlines': 1, 'china-eastern-airlines': 1, 'china-sou

In [14]:
def content_scraper(airline_name,max_page):
    url='https://www.airlinequality.com/lounge-reviews/{}/page/{}/?sortby=post_date%3ADesc&pagesize=100'
    res=[]
    for i in range(1,int(max_page)+1):
        formatted_url=url.format(airline_name,i)
        response = requests.get(formatted_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        review_list=soup.find_all('article', itemprop='review')
        for review in review_list:
            r={}
            r['title']=review.find('h2').get_text(strip=True) if review.find('h2') else 'No title'
            r['rating']=review.find('span',itemprop='ratingValue').get_text(strip=True) if review.find('span',itemprop='ratingValue') else 'No rating'
            customer_status=review.find('h3',class_='text_sub_header userStatusWrapper').get_text(strip=True)
            na_match=re.search(r'\(([^)]+)\)', customer_status)
            if na_match:
                r['nationality']=na_match.group(1)
            date_match=re.search(r'\d+\w*\s+[A-Za-z]+\s+\d{4}', customer_status)
            if date_match:
                r['date']=date_match.group(0)
            r['content']=review.find('div',class_='text_content').get_text(strip=True) if review.find('div',class_='text_content') else 'No content'
            for tr in review.find_all('tr'):
                detail_name = tr.find('td', class_='review-rating-header').text.strip()
                detail_value_container = tr.find('td', class_='review-value') or tr.find('td', class_='review-rating-stars stars')
                if detail_value_container:
                    stars = detail_value_container.find_all('span', class_='fill')
                    if stars:
                        detail_value = len(stars)
                    else:
                        detail_value = detail_value_container.text.strip()
                    r[detail_name] = detail_value
            res.append(r)
    return res

In [15]:
sample_dic={'adria-airways': 1, 'aegean-airlines': 1, 'aer-lingus': 1}
sample_res_dataframe={}
for sample_name,sample_page in sample_dic.items():
    res=content_scraper(sample_name, sample_page)
    sample_df=pd.DataFrame(res)
    sample_df['lounge']=sample_name
    sample_res_dataframe[sample_name]=sample_df
print(sample_res_dataframe)

{'adria-airways':                               title rating nationality                date  \
0    "selection of drinks was good"      7     Austria  11th February 2017   
1  "decoration is fresh and modern"      3       Spain      9th March 2014   

                                             content      Lounge Name  \
0  ❎Unverified|  The Adria Airways Business Loung...  Business Lounge   
1  There are 2 separate areas with arm chairs and...              NaN   

             Airport  Type Of Lounge     Date Visit Type Of Traveller  \
0  Ljubljana Airport  Business Class  December 2016          Business   
1                NaN  Business Class            NaN          Business   

   Comfort  Cleanliness  Bar & Beverages  Catering  Washrooms  \
0        5            4                5         1          4   
1        3            4                3         3          3   

   Wifi Connectivity  Staff Service Recommended         lounge  
0                  3              4         ye

In [20]:
final_lounges_data={}
for lounge_name,lounge_page in lounge_page.items():
    res=content_scraper(lounge_name, lounge_page)
    lounge_df=pd.DataFrame(res)
    lounge_df['Lounge']=lounge_name
    final_lounges_data[lounge_name]=lounge_df
print(len(final_lounges_data))

154


In [21]:
lounge_dt=pd.concat(final_lounges_data.values(),ignore_index=True)
lounge_dt.head()

,title,rating,nationality,date,content,Lounge Name,Airport,Type Of Lounge,Date Visit,Type Of Traveller,Comfort,Cleanliness,Bar & Beverages,Catering,Washrooms,Wifi Connectivity,Staff Service,Recommended,Lounge,Type Of Lounge Other
0,"""selection of drinks was good""",7,Austria,11th February 2017,❎Unverified| The Adria Airways Business Loung...,Business Lounge,Ljubljana Airport,Business Class,December 2016,Business,5.0,4.0,5,1,4.0,3,4.0,yes,adria-airways,NaN
1,"""decoration is fresh and modern""",3,Spain,9th March 2014,There are 2 separate areas with arm chairs and...,NaN,NaN,Business Class,NaN,Business,3.0,4.0,3,3,3.0,2,2.0,yes,adria-airways,NaN
2,"""the food options are good""",8,Portugal,8th January 2022,"Not Verified| The lounge is spacious, good lig...",Aegean Business Lounge,Athens Airport,Frequent Flyer,January 2022,NaN,5.0,4.0,4,4,NaN,5,4.0,yes,aegean-airlines,NaN
3,"""staff very polite and helpful""",6,Greece,5th March 2020,"✅Trip Verified| The lounge is often crowded, ...",NaN,Athens Airport,Business Class,March 2020,Business,3.0,4.0,4,2,3.0,4,5.0,yes,aegean-airlines,NaN
4,"""staff is friendly and courteous""",10,Germany,22nd September 2019,"✅Trip Verified| Modern lounge, good size, at ...",Aegean Lounge,Athens Airport,Business Class,September 2019,Business,5.0,5.0,5,5,4.0,5,5.0,yes,aegean-airlines,NaN


In [22]:
lounge_dt.columns

Index(['title', 'rating', 'nationality', 'date', 'content', 'Lounge Name',
       'Airport', 'Type Of Lounge', 'Date Visit', 'Type Of Traveller',
       'Comfort', 'Cleanliness', 'Bar & Beverages', 'Catering', 'Washrooms',
       'Wifi Connectivity', 'Staff Service', 'Recommended', 'Lounge',
       'Type Of Lounge Other'],
      dtype='object')

In [23]:
raw_lounge_dt=lounge_dt.loc[:,['Lounge Name','Lounge','title', 'rating', 'nationality', 'date', 'content',
       'Airport', 'Type Of Lounge', 'Date Visit', 'Type Of Traveller',
       'Comfort', 'Cleanliness', 'Bar & Beverages', 'Catering', 'Washrooms',
       'Wifi Connectivity', 'Staff Service', 'Recommended',
       'Type Of Lounge Other']]
raw_lounge_dt.head()

,Lounge Name,Lounge,title,rating,nationality,date,content,Airport,Type Of Lounge,Date Visit,Type Of Traveller,Comfort,Cleanliness,Bar & Beverages,Catering,Washrooms,Wifi Connectivity,Staff Service,Recommended,Type Of Lounge Other
0,Business Lounge,adria-airways,"""selection of drinks was good""",7,Austria,11th February 2017,❎Unverified| The Adria Airways Business Loung...,Ljubljana Airport,Business Class,December 2016,Business,5.0,4.0,5,1,4.0,3,4.0,yes,NaN
1,NaN,adria-airways,"""decoration is fresh and modern""",3,Spain,9th March 2014,There are 2 separate areas with arm chairs and...,NaN,Business Class,NaN,Business,3.0,4.0,3,3,3.0,2,2.0,yes,NaN
2,Aegean Business Lounge,aegean-airlines,"""the food options are good""",8,Portugal,8th January 2022,"Not Verified| The lounge is spacious, good lig...",Athens Airport,Frequent Flyer,January 2022,NaN,5.0,4.0,4,4,NaN,5,4.0,yes,NaN
3,NaN,aegean-airlines,"""staff very polite and helpful""",6,Greece,5th March 2020,"✅Trip Verified| The lounge is often crowded, ...",Athens Airport,Business Class,March 2020,Business,3.0,4.0,4,2,3.0,4,5.0,yes,NaN
4,Aegean Lounge,aegean-airlines,"""staff is friendly and courteous""",10,Germany,22nd September 2019,"✅Trip Verified| Modern lounge, good size, at ...",Athens Airport,Business Class,September 2019,Business,5.0,5.0,5,5,4.0,5,5.0,yes,NaN


In [26]:
raw_lounge_dt.to_csv('raw_lounge_data.csv',index=False)